In [ ]:
import os
import shutil
import random
from tqdm import tqdm
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

# Define source directory paths
source_dirs = {
    "Demented": [
        "C:\\Users\\HP\\Downloads\\Dataset_Alzheimer\\Mild_Demented",
        "C:\\Users\\HP\\Downloads\\Dataset_Alzheimer\\Moderate_Demented",
        "C:\\Users\\HP\\Downloads\\Dataset_Alzheimer\\Very_Mild_Demented"
    ],
    "Non-Demented": ["C:\\Users\\HP\\Downloads\\Dataset_Alzheimer\\Non_Demented"]
}

# Define target directories
new_base_dir = "C:\\Users\\HP\\Downloads\\Classification_Alzheimer_new"
train_dir = os.path.join(new_base_dir, "train")
test_dir = os.path.join(new_base_dir, "test")
train_demented_dir = os.path.join(train_dir, "Demented")
train_nondemented_dir = os.path.join(train_dir, "Non-Demented")
test_demented_dir = os.path.join(test_dir, "Demented")
test_nondemented_dir = os.path.join(test_dir, "Non-Demented")

# Create directories if they don't exist
for dir_path in [train_demented_dir, train_nondemented_dir, test_demented_dir, test_nondemented_dir]:
    os.makedirs(dir_path, exist_ok=True)

# Function to shuffle and split data
def shuffle_and_split_data(source_dirs, train_dir, test_dir, split_ratio=0.8):
    all_images = []
    valid_extensions = (".jpg", ".png", ".jpeg")

    for source_dir in source_dirs:
        for sub_dir in (source_dir if isinstance(source_dir, list) else [source_dir]):
            for img_file in os.listdir(sub_dir):
                if img_file.lower().endswith(valid_extensions):
                    all_images.append(os.path.join(sub_dir, img_file))

    # Shuffle images
    random.shuffle(all_images)

    # Split data
    split_point = int(len(all_images) * split_ratio)
    train_images, test_images = all_images[:split_point], all_images[split_point:]

    # Copy images
    for img_path in tqdm(train_images, desc=f'Copying to {train_dir}'):
        shutil.copy(img_path, train_dir)
    
    for img_path in tqdm(test_images, desc=f'Copying to {test_dir}'):
        shutil.copy(img_path, test_dir)
    
    return train_images, test_images

# Split and copy data
shuffle_and_split_data(source_dirs["Demented"], train_demented_dir, test_demented_dir)
shuffle_and_split_data(source_dirs["Non-Demented"], train_nondemented_dir, test_nondemented_dir)

print("Dataset organized and split successfully!")

# Feature extraction using VGG16
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
feature_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("block5_pool").output)

def extract_features(img_paths):
    features = []
    for img_path in tqdm(img_paths, desc='Extracting features'):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=0)
        img = img / 255.0
        feature = feature_extractor.predict(img)
        features.append(feature.flatten())
    return np.array(features)

train_demented_paths = [os.path.join(train_demented_dir, f) for f in os.listdir(train_demented_dir) if f.lower().endswith(valid_extensions)]
train_nondemented_paths = [os.path.join(train_nondemented_dir, f) for f in os.listdir(train_nondemented_dir) if f.lower().endswith(valid_extensions)]
test_demented_paths = [os.path.join(test_demented_dir, f) for f in os.listdir(test_demented_dir) if f.lower().endswith(valid_extensions)]
test_nondemented_paths = [os.path.join(test_nondemented_dir, f) for f in os.listdir(test_nondemented_dir) if f.lower().endswith(valid_extensions)]

train_demented_features = extract_features(train_demented_paths)
train_nondemented_features = extract_features(train_nondemented_paths)
test_demented_features = extract_features(test_demented_paths)
test_nondemented_features = extract_features(test_nondemented_paths)

train_features = np.vstack((train_demented_features, train_nondemented_features))
train_labels = np.hstack((np.ones(train_demented_features.shape[0]), np.zeros(train_nondemented_features.shape[0])))
test_features = np.vstack((test_demented_features, test_nondemented_features))
test_labels = np.hstack((np.ones(test_demented_features.shape[0]), np.zeros(test_nondemented_features.shape[0])))

print("Features extracted successfully!")


In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

# Feature extraction using VGG16
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
feature_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("block5_pool").output)

# Define valid extensions
valid_extensions = (".jpg", ".png", ".jpeg")

# Collect image paths
train_demented_paths = [os.path.join(train_demented_dir, f) for f in os.listdir(train_demented_dir) if f.lower().endswith(valid_extensions)]
tsrain_nondemented_paths = [os.path.join(train_nondemented_dir, f) for f in os.listdir(train_nondemented_dir) if f.lower().endswith(valid_extensions)]
test_demented_paths = [os.path.join(test_demented_dir, f) for f in os.listdir(test_demented_dir) if f.lower().endswith(valid_extensions)]
test_nondemented_paths = [os.path.join(test_nondemented_dir, f) for f in os.listdir(test_nondemented_dir) if f.lower().endswith(valid_extensions)]

def extract_features(img_paths):
    features = []
    for img_path in tqdm(img_paths, desc='Extracting features'):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        img = np.expand_dims(img, axis=0)
        img = img / 255.0
        feature = feature_extractor.predict(img)
        features.append(feature.flatten())
    return np.array(features)

train_demented_features = extract_features(train_demented_paths)
train_nondemented_features = extract_features(train_nondemented_paths)
test_demented_features = extract_features(test_demented_paths)
test_nondemented_features = extract_features(test_nondemented_paths)

train_features = np.vstack((train_demented_features, train_nondemented_features))
train_labels = np.hstack((np.ones(train_demented_features.shape[0]), np.zeros(train_nondemented_features.shape[0])))
test_features = np.vstack((test_demented_features, test_nondemented_features))
test_labels = np.hstack((np.ones(test_demented_features.shape[0]), np.zeros(test_nondemented_features.shape[0])))

print("Features extracted successfully!")


Extracting features:   0%|                                                                    | 0/2560 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


Extracting features:   0%|                                                          | 1/2560 [00:01<1:19:13,  1.86s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


Extracting features:   0%|                                                          | 2/2560 [00:03<1:20:07,  1.88s/it]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 753ms/step


Extracting features: 100%|██████████████████████████████████████████████████████████▉| 639/640 [08:41<00:00,  1.23it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 752ms/step


Extracting features: 100%|███████████████████████████████████████████████████████████| 640/640 [08:42<00:00,  1.23it/s]


Features extracted successfully!


In [4]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the SVM model with a linear kernel
svm_model = SVC(kernel='linear')

# Train the SVM model
svm_model.fit(train_features, train_labels)

# Make predictions on the test set
test_predictions = svm_model.predict(test_features)

# Calculate accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Generate a classification report
report = classification_report(test_labels, test_predictions, target_names=['Non-Demented', 'Demented'])
print("Classification Report:\n", report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 98.52%
Classification Report:
               precision    recall  f1-score   support

Non-Demented       0.99      0.98      0.99       640
    Demented       0.98      0.99      0.99       640

    accuracy                           0.99      1280
   macro avg       0.99      0.99      0.99      1280
weighted avg       0.99      0.99      0.99      1280

Confusion Matrix:
 [[630  10]
 [  9 631]]


In [8]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define valid extensions
valid_extensions = (".jpg", ".png", ".jpeg")

# Feature extraction using VGG16
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
feature_extractor = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("block5_pool").output)

# Define your directories (make sure these are correctly set)
train_demented_dir = "C:\\Users\\HP\\Downloads\\Classification_Alzheimer_new\\train\\Demented"
train_nondemented_dir = "C:\\Users\\HP\\Downloads\\Classification_Alzheimer_new\\train\\Non-Demented"
test_demented_dir = "C:\\Users\\HP\\Downloads\\Classification_Alzheimer_new\\test\\Demented"
test_nondemented_dir = "C:\\Users\\HP\\Downloads\\Classification_Alzheimer_new\\test\\Non-Demented"

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Collect image paths
train_demented_paths = [os.path.join(train_demented_dir, f) for f in os.listdir(train_demented_dir) if f.lower().endswith(valid_extensions)]
train_nondemented_paths = [os.path.join(train_nondemented_dir, f) for f in os.listdir(train_nondemented_dir) if f.lower().endswith(valid_extensions)]
test_demented_paths = [os.path.join(test_demented_dir, f) for f in os.listdir(test_demented_dir) if f.lower().endswith(valid_extensions)]
test_nondemented_paths = [os.path.join(test_nondemented_dir, f) for f in os.listdir(test_nondemented_dir) if f.lower().endswith(valid_extensions)]

def augment_and_extract_features(img_paths, batch_size=32):
    features = []
    num_batches = len(img_paths) // batch_size + 1

    for i in tqdm(range(num_batches), desc='Extracting features'):
        batch_paths = img_paths[i * batch_size:(i + 1) * batch_size]
        batch_images = []

        for img_path in batch_paths:
            img = cv2.imread(img_path)
            img = cv2.resize(img, (224, 224))
            img = img / 255.0
            
            # Apply augmentation
            img = datagen.random_transform(img)
            batch_images.append(img)
        
        batch_images = np.array(batch_images)
        if len(batch_images) == 0:
            continue
        
        batch_features = feature_extractor.predict(batch_images)

        for feature in batch_features:
            features.append(feature.flatten())
    
    return np.array(features)

train_demented_features = augment_and_extract_features(train_demented_paths)
train_nondemented_features = augment_and_extract_features(train_nondemented_paths)
test_demented_features = augment_and_extract_features(test_demented_paths)
test_nondemented_features = augment_and_extract_features(test_nondemented_paths)

train_features = np.vstack((train_demented_features, train_nondemented_features))
train_labels = np.hstack((np.ones(train_demented_features.shape[0]), np.zeros(train_nondemented_features.shape[0])))
test_features = np.vstack((test_demented_features, test_nondemented_features))
test_labels = np.hstack((np.ones(test_demented_features.shape[0]), np.zeros(test_nondemented_features.shape[0])))

print("Features extracted and augmented successfully!")


Extracting features:   0%|                                                                      | 0/81 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step


Extracting features:   1%|▊                                                             | 1/81 [00:08<11:01,  8.26s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step


Extracting features:   2%|█▌                                                            | 2/81 [00:19<12:57,  9.84s/it]

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step


Extracting features:  90%|███████████████████████████████████████████████████████▏     | 19/21 [03:21<00:21, 10.82s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step


Extracting features: 100%|█████████████████████████████████████████████████████████████| 21/21 [03:31<00:00, 10.08s/it]


Features extracted and augmented successfully!


In [9]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize the SVM model with a linear kernel
svm_model = SVC(kernel='linear')

# Train the SVM model
svm_model.fit(train_features, train_labels)

# Make predictions on the test set
test_predictions = svm_model.predict(test_features)

# Calculate accuracy
accuracy = accuracy_score(test_labels, test_predictions)
print(f"Accuracy: {accuracy * 100:.2f}%")

# Generate a classification report
report = classification_report(test_labels, test_predictions, target_names=['Non-Demented', 'Demented'])
print("Classification Report:\n", report)

# Generate a confusion matrix
conf_matrix = confusion_matrix(test_labels, test_predictions)
print("Confusion Matrix:\n", conf_matrix)


Accuracy: 70.47%
Classification Report:
               precision    recall  f1-score   support

Non-Demented       0.70      0.71      0.71       640
    Demented       0.71      0.70      0.70       640

    accuracy                           0.70      1280
   macro avg       0.70      0.70      0.70      1280
weighted avg       0.70      0.70      0.70      1280

Confusion Matrix:
 [[453 187]
 [191 449]]
